# Cognitive Age Prediction with Structural and Functional MRI

## Structural MRI

In [1]:
from camcan.utils import get_area, get_thickness

In [2]:
import os
from os.path import isdir, join

import numpy as np
import pandas as pd

N_CORTICAL_FEATURES = 2562

# prepare freesurfer data
STRUCTURAL_DATA = '/home/okozyn/Projects/inria/camcan_analysis/data/structural'
VOLUME_FILE = 'aseg.csv'

subjects = tuple(d for d in os.listdir(STRUCTURAL_DATA) if isdir(join(STRUCTURAL_DATA, d)))

area_data = None
thickness_data = None
volume_data = None

area_failed = []
thickness_failed = []
volume_failed = []

for s in subjects:
    subject_dir = join(STRUCTURAL_DATA, s)
    
    try:
        t_area = get_area(subject_dir, n_points=N_CORTICAL_FEATURES)
    except FileNotFoundError:
        area_failed.append(s)

    try:
        t_thickness = get_thickness(subject_dir, n_points=N_CORTICAL_FEATURES)
    except FileNotFoundError:
        volume_failed.append(s)

    try:
        volume = pd.read_csv(join(subject_dir, VOLUME_FILE), index_col=0)
    except FileNotFoundError:
        volume_failed.append(s)

    if area_data is None:
        area_data = pd.DataFrame(index=subjects, columns=np.arange(start=0, stop=2562), dtype=float)
        area_data.loc[s] = t_area[:2562]
    else:
        area_data.loc[s] = t_area[:2562]
    
    if thickness_data is None:
        thickness_data = pd.DataFrame(index=subjects, columns=np.arange(start=0, stop=2562), dtype=float)
        thickness_data.loc[s] = t_thickness[:2562]
    else:
        thickness_data.loc[s] = t_thickness[:2562]

    if volume_data is None:
        volume_data = volume
    else:
        volume_data = pd.concat([volume_data, volume])

print('Failed to load area data for\n', area_failed)
print('Failed to load thickness data for\n', thickness_failed)
print('Failed to load volume data for\n', volume_failed)

area_data.to_csv(join(STRUCTURAL_DATA, 'area_data.csv'))
thickness_data.to_csv(join(STRUCTURAL_DATA, 'thickness_data.csv'))
volume_data.to_csv(join(STRUCTURAL_DATA, 'volume_data.csv'))

/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/nibabel/keywordonly.py:17: ResourceWarning: unclosed file <_io.BufferedReader name='/home/okozyn/Projects/inria/camcan_analysis/data/structural/CC210023/lh.area.mgh'>
  return func(*args, **kwargs)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/nibabel/keywordonly.py:17: ResourceWarning: unclosed file <_io.BufferedReader name='/home/okozyn/Projects/inria/camcan_analysis/data/structural/CC210023/rh.area.mgh'>
  return func(*args, **kwargs)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/nibabel/keywordonly.py:17: ResourceWarning: unclosed file <_io.BufferedReader name='/home/okozyn/Projects/inria/camcan_analysis/data/structural/CC210023/rh.thickness.mgh'>
  return func(*args, **kwargs)
/home/okozyn/anaconda3/envs/mne/lib/python3.6/site-packages/nibabel/keywordonly.py:17: ResourceWarning: unclosed file <_io.BufferedReader name='/home/okozyn/Projects/inria/camcan_analysis/data/structural/CC210023/lh.thick

Failed to load area data for
 []
Failed to load thickness data for
 []
Failed to load volume data for
 []


In [104]:
area_data.to_pickle(join(STRUCTURAL_DATA, 'area_data.gzip'), compression='gzip')

## Functional MRI

In [17]:
import os
from os.path import join, isdir

import joblib as jb
import numpy as np
import pandas as pd

# load connectivity matrices
ATLASES = ['msdl', 'basc064', 'basc122', 'basc197']
# path for the different kind of connectivity matrices
CONNECTIVITY_KIND = ['correlation', 'partial correlation', 'tangent']
CONNECTIVITY_DATA_DIR = '/home/okozyn/Projects/inria/camcan_analysis/data/connectivity'
OUT_DIR = '/home/okozyn/Projects/inria/camcan_analysis/data/connectivity'

subjects = tuple(d[4:] for d in os.listdir(CONNECTIVITY_DATA_DIR) if isdir(join(CONNECTIVITY_DATA_DIR, d)))
connect_data = None
connect_failed = []
sel_atlas = ATLASES[1]
sel_connect = CONNECTIVITY_KIND[0]

print(f'Found {len(subjects)} subjects')

for s in subjects:
    file_path = join(CONNECTIVITY_DATA_DIR,
                     f'sub-{s}/{sel_atlas}/{sel_connect}/sub-{s}_task-Rest_confounds.pkl')
    try:
        with open(file_path, 'rb') as f:
            connect_matrix = jb.load(f)
    except OSError:
        print(f'Cannot find connectivity file {file_path} for subject {s}')
        connect_failed.append(s)

    if connect_data is None:
        connect_data = pd.DataFrame(index=subjects, columns=np.arange(start=0, stop=len(connect_matrix)),
                                         dtype=float)
        # save data to for this subject and apply Fisher's transform
        connect_data.loc[s] = np.arctanh(connect_matrix)
    else:
        # save data to for this subject and apply Fisher's transform
        connect_data.loc[s] = np.arctanh(connect_matrix)

print('Failed to load connectivity data for\n', connect_failed)

connect_data.to_pickle(join(OUT_DIR, 'connect_data.gzip'), compression='gzip')

Found 3 subjects
Failed to load connectivity data for
 []


scipy stats for Fichers transform
https://en.wikipedia.org/wiki/Fisher_transformation

This is just a tanh function

In [28]:
from itertools import product
meas = ('thickness', 'area')
hemi = ('lh', 'rh')

for h, m in product(hemi, meas):
    print(h, m)

lh thickness
lh area
rh thickness
rh area


In [1]:
from camcan import utils

In [35]:
import nibabel as nb
import numpy as np

img = nb.load(out_files['lh_thickness_file'])
in_data = img.get_data().squeeze()
# column vector
vectorized_data = in_data[:, np.newaxis]

In [36]:
t = np.genfromtxt(out_files['aseg_file'], skip_header=1, delimiter=',')[1:]

In [37]:
out_files.keys()

dict_keys(['lh_thickness_file', 'lh_area_file', 'rh_thickness_file', 'rh_area_file', 'aseg_file'])

In [38]:
t.shape

(66,)

In [33]:
vectorized_data.sort()
np.sum(vectorized_data > 0)

149600

In [39]:
img.get_data().shape

(163842, 1, 1)

In [ ]:
d = [d for d in os.listdir('./') if isdir(d) and 'CC' in d]  

In [2]:
from camcan.preprocessing.structural import get_cortex_data

In [4]:
subjects_dir = "/home/okozyn/Projects/inria/camcan_analysis/data/freesurfer"
subject = "CC620466"
out_dir = "/home/okozyn/Projects/inria/camcan_analysis/data/debug"

get_cortex_data(subjects_dir, subject, out_dir)

INFO:root:nsubjects = 1
tmpdir is /home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/tmp.mris_preproc.8801
/home/okozyn/Projects/inria/camcan_analysis/notebook/mri
Log file is /home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/lh.thickness.mris_preproc.log
ven  1 mar 2019, 15.58.47, CET
setenv SUBJECTS_DIR /home/okozyn/Projects/inria/camcan_analysis/data/freesurfer
cd /home/okozyn/Projects/inria/camcan_analysis/notebook/mri
/home/okozyn/Programs/freesurfer/bin/mris_preproc --s CC620466 --target fsaverage4 --hemi lh --meas thickness --out /home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/lh.thickness.mgh

Linux azzaro 4.15.0-45-generic #48-Ubuntu SMP Tue Jan 29 16:28:13 UTC 2019 x86_64 x86_64 x86_64 GNU/Linux
$Id: mris_preproc,v 1.76.2.1 2016/08/12 17:55:45 zkaufman Exp $
freesurfer-Linux-centos6_x86_64-stable-pub-v6.0.0-2beb96c
tmpdir is /home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/tmp.mris_preproc.8801
Src lh fsaverage4.sphere

ERROR:root:Non zero return code: 1. Bash: b'', cmd: mris_preproc --s CC620466 --target fsaverage4 --hemi rh --meas thickness --out /home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/rh.thickness.mgh
INFO:root:nsubjects = 1
INFO: turning on jacobican correction
tmpdir is /home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/tmp.mris_preproc.9016
/home/okozyn/Projects/inria/camcan_analysis/notebook/mri
Log file is /home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/rh.area.mris_preproc.log
ven  1 mar 2019, 15.58.48, CET
setenv SUBJECTS_DIR /home/okozyn/Projects/inria/camcan_analysis/data/freesurfer
cd /home/okozyn/Projects/inria/camcan_analysis/notebook/mri
/home/okozyn/Programs/freesurfer/bin/mris_preproc --s CC620466 --target fsaverage4 --hemi rh --meas area --out /home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/rh.area.mgh

Linux azzaro 4.15.0-45-generic #48-Ubuntu SMP Tue Jan 29 16:28:13 UTC 2019 x86_64 x86_64 x86_64 GNU/Linux
$Id: mr

{'lh_thickness_file': '/home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/lh.thickness.mgh',
 'lh_area_file': '/home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/lh.area.mgh',
 'rh_thickness_file': '/home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/rh.thickness.mgh',
 'rh_area_file': '/home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/rh.area.mgh'}